# Cycles

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder 
from sklearn.model_selection import train_test_split
from sklearn import metrics
import time
import matplotlib.pyplot as plt

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement=True
session = InteractiveSession(config=config)

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2060, pci bus id: 0000:26:00.0, compute capability: 7.5



# Constants

In [3]:
PATH = "data/"
BATCH_SIZE = 16
EPOCHS = 500
USER_RANGE = range(1, 154)
RANDOM_STATE = 11235
G3 = 29.4
SEQUENCE_LENGTH = 128
NUM_BINS = 10

# Data loading

In [4]:
def frames_to_cycles(recording_filename, data):
    # Open cycles.txt and convert cycles to fixed length frames
    cycles_filename = str(recording_filename).replace('3.txt', 'cycles.txt')
    dfcycles = pd.read_csv(cycles_filename, header = None, delimiter=',')
    datacycles = dfcycles.values
    n = datacycles[-1].shape[0]
    
    # cycles' endpoints
    endpoints = datacycles[-1]
    
    frames = None            
    for i in range(0, n-2):
        frame = data[endpoints[i] : endpoints[i+1], :]
        num_points = frame.shape[0]
        
        if num_points < SEQUENCE_LENGTH:
            rows = SEQUENCE_LENGTH - num_points
            zeros = np.zeros((rows, 3))
            frame = np.concatenate((frame, zeros))
        else:
            frame = frame[0:SEQUENCE_LENGTH]
            
        # Raw cycles still have the format of [X:128, Y:128, Z:128]
        frame = frame.reshape(-1, 384, order='F')

        if frames is None:
            frames = frame
        else:
            frames = np.concatenate((frames, frame), axis=0)        
    
    return frames

In [5]:
def read_cycle(filename, normalize):
    # Open file 3.txt - acceleremoter on the right hip
    df = pd.DataFrame({'X': [],
                       'Y': [],
                       'Z': []})

    with open(filename) as f:
        lines = list(map(lambda line: [float(x) for x in line.strip().split(',')], f.readlines()))
        df['X'] = lines[0]
        df['Y'] = lines[1]
        df['Z'] = lines[2]

    data = df.values
    data = frames_to_cycles(filename, data)

    # Normalization
    if normalize is True:
        data = data / G3
    
    return data

In [6]:
def read_cycle_session(session, normalize):
    X = None
    y = None
    for subj in Path('data/raw_data/' + session).glob('subj_*'):
        for rec in subj.glob('*'):
            curr = int(subj.name.split('_')[1])
            if X is None:
                X = read_cycle(rec.joinpath('3.txt'), normalize)
                y = np.ones(X.shape[0]) * curr
            else:
                X_curr = read_cycle(rec.joinpath('3.txt'), normalize)
                X = np.concatenate((X, X_curr), axis=0)
                y = np.concatenate((y, np.ones(X_curr.shape[0]) * curr), axis=0)
    return X, y

# Handcrafted features

In [7]:
def average_absolute_difference(x, avg_x):
    y = np.abs(x - avg_x)
    return np.mean(y)

In [8]:
def zero_crossing_rate(x):
    counter = 0
    for i in range(1,len(x)):
        if( (x[i-1] <= 0 and x[i] >0) or (x[i-1]>=0 and x[i]<0) ):
            counter = counter +1 
    return counter / len(x)

In [9]:
def feature_extraction(data):
    frames, features = np.shape(data)
    num_features = 19 + 4 * NUM_BINS
    result = np.zeros((frames, num_features))
    
    # Extract 59 features from each row    
    for frame in range(0, frames):       
        ax = data[frame, 0:128]
        ay = data[frame, 129:256]
        az = data[frame, 257:384]
        frame_data = data[frame, :].reshape(128, 3)
        am = np.sqrt(np.sum(np.power(frame_data, 2), axis=1))
        
        min_ax = np.min(ax)
        min_ay = np.min(ay)
        min_az = np.min(az)
        min_am = np.min(am)
        result[frame, 0] = min_ax
        result[frame, 1] = min_ay
        result[frame, 2] = min_az
        result[frame, 3] = min_am

        avg_ax = np.mean(ax)
        avg_ay = np.mean(ay)
        avg_az = np.mean(az)
        avg_am = np.mean(am)
        result[frame, 4] = avg_ax
        result[frame, 5] = avg_ay
        result[frame, 6] = avg_az
        result[frame, 7] = avg_am

        std_ax = np.std(ax)
        std_ay = np.std(ay)
        std_az = np.std(az)
        std_am = np.std(am)
        result[frame, 8]  = std_ax
        result[frame, 9]  = std_ay
        result[frame, 10] = std_az
        result[frame, 11] = std_am

        aad_ax = average_absolute_difference(ax, avg_ax)
        aad_ay = average_absolute_difference(ay, avg_ay)
        aad_az = average_absolute_difference(az, avg_az)
        aad_am = average_absolute_difference(am, avg_am)
        result[frame, 12] = aad_ax
        result[frame, 13] = aad_ay
        result[frame, 14] = aad_az
        result[frame, 15] = aad_am

        zcr_ax = zero_crossing_rate( ax )
        zcr_ay = zero_crossing_rate( ay )
        zcr_az = zero_crossing_rate( az )
        result[frame, 16] = zcr_ax
        result[frame, 17] = zcr_ay
        result[frame, 18] = zcr_az

        histo_ax, bin_edges = np.histogram(ax, NUM_BINS, (-1.5,1.5), density=True)
        histo_ay, bin_edges = np.histogram(ay, NUM_BINS, (-1.5,1.5), density=True)
        histo_az, bin_edges = np.histogram(az, NUM_BINS, (-1.5,1.5), density=True)
        histo_am, bin_edges = np.histogram(am, NUM_BINS, (0,2), density=True)
        for i in range(0, NUM_BINS):
            result[frame, 19 + i] = histo_ax[i]
        for i in range(0, NUM_BINS):
            result[frame, 19 + NUM_BINS+i] = histo_ay[i]
        for i in range(0, NUM_BINS):
            result[frame, 19 + 2 * NUM_BINS + i] = histo_az[i]
        for i in range(0, NUM_BINS):
            result[frame, 19 + 3 * NUM_BINS + i] = histo_am[i]
        
        num_NAN = np.count_nonzero(np.isnan(result[frame, :]))
    
        if (num_NAN != 0):
            result[frame, :] = result[frame-1, :]
        
    return result

# Neural models

In [10]:
def build_resnet(input_shape, nb_classes, file_path):
    n_feature_maps = 64
    input_layer = keras.layers.Input(input_shape)

    # BLOCK 1 
    conv_x = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=8, padding='same')(input_layer)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)
    conv_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=5, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)
    conv_z = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=3, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    # expand channels for the sum 
    shortcut_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=1, padding='same')(input_layer)
    shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
    output_block_1 = keras.layers.add([shortcut_y, conv_z])
    output_block_1 = keras.layers.Activation('relu')(output_block_1)

    # BLOCK 2 
    conv_x = keras.layers.Conv1D(filters=n_feature_maps*2, kernel_size=8, padding='same')(output_block_1)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)
    conv_y = keras.layers.Conv1D(filters=n_feature_maps*2, kernel_size=5, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)
    conv_z = keras.layers.Conv1D(filters=n_feature_maps*2, kernel_size=3, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    # expand channels for the sum 
    shortcut_y = keras.layers.Conv1D(filters=n_feature_maps*2, kernel_size=1, padding='same')(output_block_1)
    shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
    output_block_2 = keras.layers.add([shortcut_y, conv_z])
    output_block_2 = keras.layers.Activation('relu')(output_block_2)

    # BLOCK 3 
    conv_x = keras.layers.Conv1D(filters=n_feature_maps*2, kernel_size=8, padding='same')(output_block_2)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)
    conv_y = keras.layers.Conv1D(filters=n_feature_maps*2, kernel_size=5, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)
    conv_z = keras.layers.Conv1D(filters=n_feature_maps*2, kernel_size=3, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    # no need to expand channels because they are equal 
    shortcut_y = keras.layers.BatchNormalization()(output_block_2)
    output_block_3 = keras.layers.add([shortcut_y, conv_z])
    output_block_3 = keras.layers.Activation('relu')(output_block_3)

    # FINAL 
    gap_layer = keras.layers.GlobalAveragePooling1D()(output_block_3)
    output_layer = keras.layers.Dense(nb_classes, activation='softmax')(gap_layer)

    model = keras.models.Model(inputs=input_layer, outputs=output_layer)
    model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(), metrics=['categorical_accuracy'])
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50, min_lr=0.0001)

    model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='loss', save_best_only=True)
    callbacks = [reduce_lr,model_checkpoint]

    return callbacks, model

In [11]:
def build_fcn(input_shape, nb_classes, file_path):
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=128, kernel_size=8, padding='same')(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.Activation(activation='relu')(conv1)

    conv2 = keras.layers.Conv1D(filters=256, kernel_size=5, padding='same')(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.Activation('relu')(conv2)

    conv3 = keras.layers.Conv1D(128, kernel_size=3,padding='same')(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.Activation('relu')(conv3)

    gap_layer = keras.layers.GlobalAveragePooling1D()(conv3)

    output_layer = keras.layers.Dense(nb_classes, activation='softmax')(gap_layer)

    model = keras.models.Model(inputs=input_layer, outputs=output_layer)

    model.compile(loss='categorical_crossentropy', optimizer = keras.optimizers.Adam(), 
                  metrics=['categorical_accuracy'])

    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50, 
                                                  min_lr=0.0001)
    
    model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='loss', 
                                                       save_best_only=True)

    callbacks = [reduce_lr,model_checkpoint]

    return callbacks, model 

# Measurements

## Raw

### Same-day

In [12]:
X, y = read_cycle_session("session_1", normalize=False)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=RANDOM_STATE)

In [14]:
model = RandomForestClassifier(n_estimators = 100, random_state=RANDOM_STATE)
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=11235,
                       verbose=0, warm_start=False)

In [15]:
model.score(X_test, y_test)

0.947035118019574

### Cross-day

In [16]:
X, y = read_cycle_session("session_2", normalize=False)

In [17]:
model.score(X, y)

0.44587747581759557

## Raw normalized

### Same-day

In [18]:
X, y = read_cycle_session("session_1", normalize=True)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=RANDOM_STATE)

In [20]:
model = RandomForestClassifier(n_estimators = 100, random_state=RANDOM_STATE)
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=11235,
                       verbose=0, warm_start=False)

In [21]:
model.score(X_test, y_test)

0.947035118019574

## Cross-day

In [22]:
X, y = read_cycle_session("session_2", normalize=True)

In [23]:
model.score(X, y)

0.44587747581759557

## Handcrafted

### Same-day

In [24]:
X, y = read_cycle_session("session_1", normalize=False)

In [25]:
X_feat = feature_extraction(X)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X_feat, y, test_size=0.2, random_state=RANDOM_STATE)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=RANDOM_STATE)

In [27]:
model = RandomForestClassifier(n_estimators = 100, random_state=RANDOM_STATE)
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=11235,
                       verbose=0, warm_start=False)

In [28]:
model.score(X_test, y_test)

0.9291882556131261

### Cross-day

In [29]:
X, y = read_cycle_session("session_2", normalize=False)

In [30]:
X_feat = feature_extraction(X)

In [31]:
model.score(X_feat, y)

0.24804237678489174

## Handcrafted normalized

### Same-day

In [32]:
X, y = read_cycle_session("session_1", normalize=True)

In [33]:
X_feat = feature_extraction(X)

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X_feat, y, test_size=0.2, random_state=RANDOM_STATE)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=RANDOM_STATE)

In [35]:
model = RandomForestClassifier(n_estimators = 100, random_state=RANDOM_STATE)
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=11235,
                       verbose=0, warm_start=False)

In [36]:
model.score(X_test, y_test)

0.9050086355785838

### Cross-day

In [37]:
X, y = read_cycle_session("session_2", normalize=True)

In [38]:
X_feat = feature_extraction(X)

In [39]:
model.score(X_feat, y)

0.204398894518655

## ResNet

### Same-day

In [40]:
X, y = read_cycle_session("session_1", normalize=False)

In [41]:
enc = OneHotEncoder()
enc.fit(y.reshape(-1,1))
y = enc.transform(y.reshape(-1, 1)).toarray()
X = X.reshape(-1, 128, 3)

In [42]:
cb, model = build_resnet((128, 3), 153, "zju_cycle_raw_resnet_500.hdf5")

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=RANDOM_STATE)

In [44]:
mini_batch_size = int(min(X_train.shape[0]/10, BATCH_SIZE))
start_time = time.time()
hist = model.fit(X_train, y_train, 
                      batch_size=mini_batch_size, 
                      epochs=EPOCHS,
                      verbose=True, 
                      validation_data=(X_val, y_val), 
                      callbacks=cb)
duration = time.time() - start_time
y_pred = model.predict(X_test)
keras.backend.clear_session()

Train on 5210 samples, validate on 1737 samples
Epoch 1/500
5210/5210 [==============================] - 15s 3ms/sample - loss: 3.8605 - categorical_accuracy: 0.1854 - val_loss: 5.9537 - val_categorical_accuracy: 0.0363
Epoch 2/500
5210/5210 [==============================] - 4s 821us/sample - loss: 1.9682 - categorical_accuracy: 0.6061 - val_loss: 1.5978 - val_categorical_accuracy: 0.6183
Epoch 3/500
5210/5210 [==============================] - 4s 822us/sample - loss: 0.9282 - categorical_accuracy: 0.8497 - val_loss: 0.6220 - val_categorical_accuracy: 0.8912
Epoch 4/500
5210/5210 [==============================] - 4s 817us/sample - loss: 0.5019 - categorical_accuracy: 0.9259 - val_loss: 0.4106 - val_categorical_accuracy: 0.9355
Epoch 5/500
5210/5210 [==============================] - 4s 824us/sample - loss: 0.3095 - categorical_accuracy: 0.9530 - val_loss: 0.6107 - val_categorical_accuracy: 0.8417
Epoch 6/500
5210/5210 [==============================] - 4s 832us/sample - loss: 0.2225 

Epoch 48/500
5210/5210 [==============================] - 4s 777us/sample - loss: 0.0048 - categorical_accuracy: 0.9987 - val_loss: 0.1319 - val_categorical_accuracy: 0.9718
Epoch 49/500
5210/5210 [==============================] - 4s 791us/sample - loss: 0.0349 - categorical_accuracy: 0.9900 - val_loss: 0.2206 - val_categorical_accuracy: 0.9505
Epoch 50/500
5210/5210 [==============================] - 4s 761us/sample - loss: 0.0172 - categorical_accuracy: 0.9950 - val_loss: 0.1815 - val_categorical_accuracy: 0.9660
Epoch 51/500
5210/5210 [==============================] - 4s 800us/sample - loss: 0.0030 - categorical_accuracy: 0.9998 - val_loss: 0.1246 - val_categorical_accuracy: 0.9770
Epoch 52/500
5210/5210 [==============================] - 4s 768us/sample - loss: 0.0047 - categorical_accuracy: 0.9990 - val_loss: 0.1127 - val_categorical_accuracy: 0.9758
Epoch 53/500
5210/5210 [==============================] - 4s 791us/sample - loss: 0.0060 - categorical_accuracy: 0.9983 - val_loss

Epoch 95/500
5210/5210 [==============================] - 4s 794us/sample - loss: 8.4219e-04 - categorical_accuracy: 0.9998 - val_loss: 0.1235 - val_categorical_accuracy: 0.9770
Epoch 96/500
5210/5210 [==============================] - 4s 805us/sample - loss: 0.0042 - categorical_accuracy: 0.9990 - val_loss: 0.1400 - val_categorical_accuracy: 0.9741
Epoch 97/500
5210/5210 [==============================] - 4s 808us/sample - loss: 0.0088 - categorical_accuracy: 0.9987 - val_loss: 0.1852 - val_categorical_accuracy: 0.9620
Epoch 98/500
5210/5210 [==============================] - 4s 809us/sample - loss: 0.0035 - categorical_accuracy: 0.9990 - val_loss: 0.1385 - val_categorical_accuracy: 0.9764
Epoch 99/500
5210/5210 [==============================] - 4s 804us/sample - loss: 7.4161e-04 - categorical_accuracy: 0.9998 - val_loss: 0.1316 - val_categorical_accuracy: 0.9781
Epoch 100/500
5210/5210 [==============================] - 4s 815us/sample - loss: 5.2508e-04 - categorical_accuracy: 1.00

5210/5210 [==============================] - 4s 802us/sample - loss: 0.0037 - categorical_accuracy: 0.9992 - val_loss: 0.1993 - val_categorical_accuracy: 0.9632
Epoch 142/500
5210/5210 [==============================] - 4s 805us/sample - loss: 0.0117 - categorical_accuracy: 0.9969 - val_loss: 0.1830 - val_categorical_accuracy: 0.9649
Epoch 143/500
5210/5210 [==============================] - 4s 804us/sample - loss: 0.0033 - categorical_accuracy: 0.9994 - val_loss: 0.1382 - val_categorical_accuracy: 0.9747
Epoch 144/500
5210/5210 [==============================] - 4s 802us/sample - loss: 0.0012 - categorical_accuracy: 0.9996 - val_loss: 0.1561 - val_categorical_accuracy: 0.9758
Epoch 145/500
5210/5210 [==============================] - 4s 809us/sample - loss: 0.0060 - categorical_accuracy: 0.9983 - val_loss: 0.1602 - val_categorical_accuracy: 0.9712
Epoch 146/500
5210/5210 [==============================] - 4s 815us/sample - loss: 0.0029 - categorical_accuracy: 0.9990 - val_loss: 0.1274

5210/5210 [==============================] - 4s 817us/sample - loss: 1.8306e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1310 - val_categorical_accuracy: 0.9810
Epoch 188/500
5210/5210 [==============================] - 4s 794us/sample - loss: 1.0716e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1291 - val_categorical_accuracy: 0.9822
Epoch 189/500
5210/5210 [==============================] - 4s 797us/sample - loss: 6.7658e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1274 - val_categorical_accuracy: 0.9822
Epoch 190/500
5210/5210 [==============================] - 4s 807us/sample - loss: 6.0150e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1299 - val_categorical_accuracy: 0.9827
Epoch 191/500
5210/5210 [==============================] - 4s 796us/sample - loss: 5.7140e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1304 - val_categorical_accuracy: 0.9827
Epoch 192/500
5210/5210 [==============================] - 4s 796us/sample - loss: 3.6231e-05 - categorical_accuracy: 1

...

Epoch 279/500
5210/5210 [==============================] - 4s 782us/sample - loss: 2.3365e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1258 - val_categorical_accuracy: 0.9827
Epoch 280/500
5210/5210 [==============================] - 4s 797us/sample - loss: 2.0463e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1263 - val_categorical_accuracy: 0.9839
Epoch 281/500
5210/5210 [==============================] - 4s 774us/sample - loss: 3.9970e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1266 - val_categorical_accuracy: 0.9833
Epoch 282/500
5210/5210 [==============================] - 4s 809us/sample - loss: 3.3033e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1263 - val_categorical_accuracy: 0.9827
Epoch 283/500
5210/5210 [==============================] - 4s 792us/sample - loss: 3.5256e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1248 - val_categorical_accuracy: 0.9845
Epoch 284/500
5210/5210 [==============================] - 4s 810us/sample - loss: 3.7653e-06 - categoric

Epoch 325/500
5210/5210 [==============================] - 4s 800us/sample - loss: 1.3884e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1291 - val_categorical_accuracy: 0.9827
Epoch 326/500
5210/5210 [==============================] - 4s 810us/sample - loss: 1.2550e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1278 - val_categorical_accuracy: 0.9839
Epoch 327/500
5210/5210 [==============================] - 4s 799us/sample - loss: 1.3540e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1266 - val_categorical_accuracy: 0.9827
Epoch 328/500
5210/5210 [==============================] - 4s 786us/sample - loss: 2.3192e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1303 - val_categorical_accuracy: 0.9822
Epoch 329/500
5210/5210 [==============================] - 4s 797us/sample - loss: 3.6070e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1346 - val_categorical_accuracy: 0.9839
Epoch 330/500
5210/5210 [==============================] - 4s 785us/sample - loss: 1.1973e-06 - categoric

Epoch 371/500
5210/5210 [==============================] - 4s 812us/sample - loss: 5.6157e-07 - categorical_accuracy: 1.0000 - val_loss: 0.1423 - val_categorical_accuracy: 0.9816
Epoch 372/500
5210/5210 [==============================] - 4s 822us/sample - loss: 3.9075e-07 - categorical_accuracy: 1.0000 - val_loss: 0.1415 - val_categorical_accuracy: 0.9810
Epoch 373/500
5210/5210 [==============================] - 4s 802us/sample - loss: 5.4779e-07 - categorical_accuracy: 1.0000 - val_loss: 0.1442 - val_categorical_accuracy: 0.9816
Epoch 374/500
5210/5210 [==============================] - 4s 791us/sample - loss: 4.9089e-07 - categorical_accuracy: 1.0000 - val_loss: 0.1424 - val_categorical_accuracy: 0.9816
Epoch 375/500
5210/5210 [==============================] - 4s 798us/sample - loss: 5.1721e-07 - categorical_accuracy: 1.0000 - val_loss: 0.1445 - val_categorical_accuracy: 0.9816
Epoch 376/500
5210/5210 [==============================] - 4s 799us/sample - loss: 7.6297e-07 - categoric

Epoch 417/500
5210/5210 [==============================] - 4s 798us/sample - loss: 3.6629e-07 - categorical_accuracy: 1.0000 - val_loss: 0.1420 - val_categorical_accuracy: 0.9827
Epoch 418/500
5210/5210 [==============================] - 4s 797us/sample - loss: 3.9489e-07 - categorical_accuracy: 1.0000 - val_loss: 0.1414 - val_categorical_accuracy: 0.9822
Epoch 419/500
5210/5210 [==============================] - 4s 797us/sample - loss: 3.0066e-07 - categorical_accuracy: 1.0000 - val_loss: 0.1397 - val_categorical_accuracy: 0.9822
Epoch 420/500
5210/5210 [==============================] - 4s 799us/sample - loss: 1.1944e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1453 - val_categorical_accuracy: 0.9822
Epoch 421/500
5210/5210 [==============================] - 4s 800us/sample - loss: 8.6503e-07 - categorical_accuracy: 1.0000 - val_loss: 0.1443 - val_categorical_accuracy: 0.9839
Epoch 422/500
5210/5210 [==============================] - 4s 792us/sample - loss: 4.7016e-07 - categoric

Epoch 463/500
5210/5210 [==============================] - 4s 798us/sample - loss: 2.2910e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1445 - val_categorical_accuracy: 0.9827
Epoch 464/500
5210/5210 [==============================] - 4s 802us/sample - loss: 2.8259e-07 - categorical_accuracy: 1.0000 - val_loss: 0.1408 - val_categorical_accuracy: 0.9839
Epoch 465/500
5210/5210 [==============================] - 4s 809us/sample - loss: 2.2224e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1402 - val_categorical_accuracy: 0.9839
Epoch 466/500
5210/5210 [==============================] - 4s 803us/sample - loss: 3.4784e-07 - categorical_accuracy: 1.0000 - val_loss: 0.1399 - val_categorical_accuracy: 0.9833
Epoch 467/500
5210/5210 [==============================] - 4s 818us/sample - loss: 2.0931e-07 - categorical_accuracy: 1.0000 - val_loss: 0.1421 - val_categorical_accuracy: 0.9839
Epoch 468/500
5210/5210 [==============================] - 4s 821us/sample - loss: 1.4543e-07 - categoric

In [45]:
metrics.accuracy_score(np.argmax(y_test, axis=-1),
                       np.argmax(model.predict(X_test), axis=-1))

0.9856073690270581

### Cross-day

In [46]:
X, y = read_cycle_session("session_2", normalize=False)

In [47]:
enc = OneHotEncoder()
enc.fit(y.reshape(-1,1))
y = enc.transform(y.reshape(-1, 1)).toarray()
X = X.reshape(-1, 128, 3)

In [48]:
y_pred = model.predict(X)
y_pred = np.argmax(y_pred, axis=1)

In [49]:
metrics.accuracy_score(np.argmax(y, axis=1), y_pred)

0.522109626900046

## ResNet normalized

### Same-day

In [50]:
X, y = read_cycle_session("session_1", normalize=True)

In [51]:
enc = OneHotEncoder()
enc.fit(y.reshape(-1,1))
y = enc.transform(y.reshape(-1, 1)).toarray()
X = X.reshape(-1, 128, 3)

In [52]:
cb, model = build_resnet((128, 3), 153, "zju_cycle_norm_resnet_500.hdf5")

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=RANDOM_STATE)

In [54]:
mini_batch_size = int(min(X_train.shape[0]/10, BATCH_SIZE))
start_time = time.time()
hist = model.fit(X_train, y_train, 
                      batch_size=mini_batch_size, 
                      epochs=EPOCHS,
                      verbose=True, 
                      validation_data=(X_val, y_val), 
                      callbacks=cb)
duration = time.time() - start_time
y_pred = model.predict(X_test)
keras.backend.clear_session()

Train on 5210 samples, validate on 1737 samples
Epoch 1/500
5210/5210 [==============================] - 6s 1ms/sample - loss: 3.7879 - categorical_accuracy: 0.2090 - val_loss: 12.3843 - val_categorical_accuracy: 0.0063
Epoch 2/500
5210/5210 [==============================] - 4s 821us/sample - loss: 1.7571 - categorical_accuracy: 0.6804 - val_loss: 20.2220 - val_categorical_accuracy: 0.0132
Epoch 3/500
5210/5210 [==============================] - 4s 839us/sample - loss: 0.7517 - categorical_accuracy: 0.8940 - val_loss: 9.1981 - val_categorical_accuracy: 0.0161
Epoch 4/500
5210/5210 [==============================] - 4s 822us/sample - loss: 0.3977 - categorical_accuracy: 0.9436 - val_loss: 22.6900 - val_categorical_accuracy: 0.0052
Epoch 5/500
5210/5210 [==============================] - 4s 834us/sample - loss: 0.2727 - categorical_accuracy: 0.9557 - val_loss: 16.5712 - val_categorical_accuracy: 0.0173
Epoch 6/500
5210/5210 [==============================] - 4s 821us/sample - loss: 0.18

Epoch 48/500
5210/5210 [==============================] - 4s 781us/sample - loss: 0.0141 - categorical_accuracy: 0.9973 - val_loss: 0.1423 - val_categorical_accuracy: 0.9672
Epoch 49/500
5210/5210 [==============================] - 4s 794us/sample - loss: 0.0029 - categorical_accuracy: 0.9996 - val_loss: 0.1270 - val_categorical_accuracy: 0.9747
Epoch 50/500
5210/5210 [==============================] - 4s 778us/sample - loss: 0.0043 - categorical_accuracy: 0.9990 - val_loss: 0.1604 - val_categorical_accuracy: 0.9695
Epoch 51/500
5210/5210 [==============================] - 4s 766us/sample - loss: 0.0040 - categorical_accuracy: 0.9990 - val_loss: 0.1898 - val_categorical_accuracy: 0.9591
Epoch 52/500
5210/5210 [==============================] - 4s 756us/sample - loss: 0.0038 - categorical_accuracy: 0.9996 - val_loss: 0.1132 - val_categorical_accuracy: 0.9781
Epoch 53/500
5210/5210 [==============================] - 4s 778us/sample - loss: 0.0106 - categorical_accuracy: 0.9979 - val_loss

Epoch 95/500
5210/5210 [==============================] - 4s 816us/sample - loss: 2.1599e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1006 - val_categorical_accuracy: 0.9822
Epoch 96/500
5210/5210 [==============================] - 4s 796us/sample - loss: 2.7623e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1018 - val_categorical_accuracy: 0.9804
Epoch 97/500
5210/5210 [==============================] - 4s 793us/sample - loss: 4.2949e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1055 - val_categorical_accuracy: 0.9793
Epoch 98/500
5210/5210 [==============================] - 4s 818us/sample - loss: 1.8921e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1050 - val_categorical_accuracy: 0.9822
Epoch 99/500
5210/5210 [==============================] - 4s 798us/sample - loss: 0.0356 - categorical_accuracy: 0.9914 - val_loss: 16.0736 - val_categorical_accuracy: 0.0938
Epoch 100/500
5210/5210 [==============================] - 4s 801us/sample - loss: 0.0155 - categorical_accuracy:

5210/5210 [==============================] - 4s 798us/sample - loss: 4.7952e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1402 - val_categorical_accuracy: 0.9793
Epoch 142/500
5210/5210 [==============================] - 4s 797us/sample - loss: 2.3917e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1280 - val_categorical_accuracy: 0.9804
Epoch 143/500
5210/5210 [==============================] - 4s 796us/sample - loss: 1.8987e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1291 - val_categorical_accuracy: 0.9781
Epoch 144/500
5210/5210 [==============================] - 4s 809us/sample - loss: 8.5669e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1261 - val_categorical_accuracy: 0.9799
Epoch 145/500
5210/5210 [==============================] - 4s 808us/sample - loss: 1.8997e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1160 - val_categorical_accuracy: 0.9816
Epoch 146/500
5210/5210 [==============================] - 4s 799us/sample - loss: 1.3466e-04 - categorical_accuracy: 1

Epoch 187/500
5210/5210 [==============================] - 4s 796us/sample - loss: 1.1500e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1387 - val_categorical_accuracy: 0.9775
Epoch 188/500
5210/5210 [==============================] - 4s 806us/sample - loss: 1.6489e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1301 - val_categorical_accuracy: 0.9804
Epoch 189/500
5210/5210 [==============================] - 4s 798us/sample - loss: 1.1580e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1249 - val_categorical_accuracy: 0.9816
Epoch 190/500
5210/5210 [==============================] - 4s 792us/sample - loss: 6.2946e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1219 - val_categorical_accuracy: 0.9845
Epoch 191/500
5210/5210 [==============================] - 4s 793us/sample - loss: 9.4940e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1201 - val_categorical_accuracy: 0.9810
Epoch 192/500
5210/5210 [==============================] - 4s 802us/sample - loss: 3.5353e-04 - categoric

...

Epoch 279/500
5210/5210 [==============================] - 4s 794us/sample - loss: 3.6107e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1211 - val_categorical_accuracy: 0.9839
Epoch 280/500
5210/5210 [==============================] - 4s 783us/sample - loss: 3.1568e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1208 - val_categorical_accuracy: 0.9839
Epoch 281/500
5210/5210 [==============================] - 4s 791us/sample - loss: 5.1872e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1196 - val_categorical_accuracy: 0.9839
Epoch 282/500
5210/5210 [==============================] - 4s 788us/sample - loss: 1.0743e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1162 - val_categorical_accuracy: 0.9822
Epoch 283/500
5210/5210 [==============================] - 4s 790us/sample - loss: 3.3628e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1142 - val_categorical_accuracy: 0.9839
Epoch 284/500
5210/5210 [==============================] - 4s 793us/sample - loss: 5.6521e-06 - categoric

Epoch 325/500
5210/5210 [==============================] - 4s 794us/sample - loss: 8.5470e-07 - categorical_accuracy: 1.0000 - val_loss: 0.1165 - val_categorical_accuracy: 0.9839
Epoch 326/500
5210/5210 [==============================] - 4s 786us/sample - loss: 1.3565e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1164 - val_categorical_accuracy: 0.9839
Epoch 327/500
5210/5210 [==============================] - 4s 791us/sample - loss: 1.5433e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1176 - val_categorical_accuracy: 0.9833
Epoch 328/500
5210/5210 [==============================] - 4s 812us/sample - loss: 8.0954e-07 - categorical_accuracy: 1.0000 - val_loss: 0.1186 - val_categorical_accuracy: 0.9827
Epoch 329/500
5210/5210 [==============================] - 4s 780us/sample - loss: 9.4279e-07 - categorical_accuracy: 1.0000 - val_loss: 0.1192 - val_categorical_accuracy: 0.9833
Epoch 330/500
5210/5210 [==============================] - 4s 812us/sample - loss: 6.9389e-07 - categoric

Epoch 371/500
5210/5210 [==============================] - 4s 784us/sample - loss: 7.9205e-07 - categorical_accuracy: 1.0000 - val_loss: 0.1314 - val_categorical_accuracy: 0.9822
Epoch 372/500
5210/5210 [==============================] - 4s 797us/sample - loss: 5.8054e-07 - categorical_accuracy: 1.0000 - val_loss: 0.1287 - val_categorical_accuracy: 0.9827
Epoch 373/500
5210/5210 [==============================] - 4s 789us/sample - loss: 3.5313e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1253 - val_categorical_accuracy: 0.9822
Epoch 374/500
5210/5210 [==============================] - 4s 793us/sample - loss: 8.2200e-07 - categorical_accuracy: 1.0000 - val_loss: 0.1255 - val_categorical_accuracy: 0.9827
Epoch 375/500
5210/5210 [==============================] - 4s 806us/sample - loss: 7.4773e-07 - categorical_accuracy: 1.0000 - val_loss: 0.1256 - val_categorical_accuracy: 0.9827
Epoch 376/500
5210/5210 [==============================] - 4s 801us/sample - loss: 6.0410e-07 - categoric

Epoch 417/500
5210/5210 [==============================] - 4s 781us/sample - loss: 5.0503e-07 - categorical_accuracy: 1.0000 - val_loss: 0.1269 - val_categorical_accuracy: 0.9827
Epoch 418/500
5210/5210 [==============================] - 4s 784us/sample - loss: 3.9188e-07 - categorical_accuracy: 1.0000 - val_loss: 0.1258 - val_categorical_accuracy: 0.9827
Epoch 419/500
5210/5210 [==============================] - 4s 768us/sample - loss: 4.0121e-07 - categorical_accuracy: 1.0000 - val_loss: 0.1234 - val_categorical_accuracy: 0.9833
Epoch 420/500
5210/5210 [==============================] - 4s 802us/sample - loss: 9.2102e-07 - categorical_accuracy: 1.0000 - val_loss: 0.1262 - val_categorical_accuracy: 0.9839
Epoch 421/500
5210/5210 [==============================] - 4s 773us/sample - loss: 4.3345e-07 - categorical_accuracy: 1.0000 - val_loss: 0.1256 - val_categorical_accuracy: 0.9845
Epoch 422/500
5210/5210 [==============================] - 4s 764us/sample - loss: 8.6195e-07 - categoric

Epoch 463/500
5210/5210 [==============================] - 4s 780us/sample - loss: 2.9179e-07 - categorical_accuracy: 1.0000 - val_loss: 0.1181 - val_categorical_accuracy: 0.9839
Epoch 464/500
5210/5210 [==============================] - 4s 800us/sample - loss: 2.3629e-07 - categorical_accuracy: 1.0000 - val_loss: 0.1181 - val_categorical_accuracy: 0.9833
Epoch 465/500
5210/5210 [==============================] - 4s 809us/sample - loss: 1.2084e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1351 - val_categorical_accuracy: 0.9833
Epoch 466/500
5210/5210 [==============================] - 4s 799us/sample - loss: 2.0495e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1334 - val_categorical_accuracy: 0.9827
Epoch 467/500
5210/5210 [==============================] - 4s 791us/sample - loss: 6.4280e-07 - categorical_accuracy: 1.0000 - val_loss: 0.1299 - val_categorical_accuracy: 0.9827
Epoch 468/500
5210/5210 [==============================] - 4s 786us/sample - loss: 3.2492e-07 - categoric

In [55]:
metrics.accuracy_score(np.argmax(y_test, axis=-1),
                       np.argmax(model.predict(X_test), axis=-1))

0.9867587795048935

### Cross-day

In [56]:
X, y = read_cycle_session("session_2", normalize=True)

In [57]:
enc = OneHotEncoder()
enc.fit(y.reshape(-1,1))
y = enc.transform(y.reshape(-1, 1)).toarray()
X = X.reshape(-1, 128, 3)

In [58]:
y_pred = model.predict(X)
y_pred = np.argmax(y_pred, axis=1)

In [59]:
metrics.accuracy_score(np.argmax(y, axis=1), y_pred)

0.5162367572547213

## FCN

### Same-day

In [71]:
X, y = read_cycle_session("session_1", normalize=False)

In [72]:
enc = OneHotEncoder()
enc.fit(y.reshape(-1,1))
y = enc.transform(y.reshape(-1, 1)).toarray()
X = X.reshape(-1, 128, 3)

In [73]:
cb, model = build_fcn((128, 3), 153, "zju_cycle_raw_fcn_500.hdf5")

In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=RANDOM_STATE)

In [75]:
mini_batch_size = int(min(X_train.shape[0]/10, BATCH_SIZE))
start_time = time.time()
hist = model.fit(X_train, y_train, 
                      batch_size=mini_batch_size, 
                      epochs=EPOCHS,
                      verbose=True, 
                      validation_data=(X_val, y_val), 
                      callbacks=cb)
duration = time.time() - start_time
y_pred = model.predict(X_test)
keras.backend.clear_session()

Train on 5210 samples, validate on 1737 samples
Epoch 1/500
5210/5210 [==============================] - 3s 573us/sample - loss: 4.5462 - categorical_accuracy: 0.0601 - val_loss: 7.0939 - val_categorical_accuracy: 0.0121
Epoch 2/500
5210/5210 [==============================] - 2s 387us/sample - loss: 3.6144 - categorical_accuracy: 0.2436 - val_loss: 3.4295 - val_categorical_accuracy: 0.1658
Epoch 3/500
5210/5210 [==============================] - 2s 367us/sample - loss: 2.7300 - categorical_accuracy: 0.4797 - val_loss: 2.5355 - val_categorical_accuracy: 0.4226
Epoch 4/500
5210/5210 [==============================] - 2s 372us/sample - loss: 1.9616 - categorical_accuracy: 0.6789 - val_loss: 1.9245 - val_categorical_accuracy: 0.5596
Epoch 5/500
5210/5210 [==============================] - 2s 351us/sample - loss: 1.3647 - categorical_accuracy: 0.8063 - val_loss: 1.3325 - val_categorical_accuracy: 0.7686
Epoch 6/500
5210/5210 [==============================] - 2s 357us/sample - loss: 0.9485

Epoch 48/500
5210/5210 [==============================] - 2s 359us/sample - loss: 0.0364 - categorical_accuracy: 0.9940 - val_loss: 0.2745 - val_categorical_accuracy: 0.9349
Epoch 49/500
5210/5210 [==============================] - 2s 355us/sample - loss: 0.0167 - categorical_accuracy: 0.9965 - val_loss: 0.1269 - val_categorical_accuracy: 0.9724
Epoch 50/500
5210/5210 [==============================] - 2s 382us/sample - loss: 0.0066 - categorical_accuracy: 0.9996 - val_loss: 0.1537 - val_categorical_accuracy: 0.9649
Epoch 51/500
5210/5210 [==============================] - 2s 365us/sample - loss: 0.0194 - categorical_accuracy: 0.9965 - val_loss: 0.3277 - val_categorical_accuracy: 0.9206
Epoch 52/500
5210/5210 [==============================] - 2s 360us/sample - loss: 0.0253 - categorical_accuracy: 0.9954 - val_loss: 0.9499 - val_categorical_accuracy: 0.7617
Epoch 53/500
5210/5210 [==============================] - 2s 377us/sample - loss: 0.0185 - categorical_accuracy: 0.9962 - val_loss

5210/5210 [==============================] - 2s 350us/sample - loss: 0.0015 - categorical_accuracy: 1.0000 - val_loss: 0.1187 - val_categorical_accuracy: 0.9747
Epoch 96/500
5210/5210 [==============================] - 2s 370us/sample - loss: 0.0011 - categorical_accuracy: 1.0000 - val_loss: 0.1131 - val_categorical_accuracy: 0.9729
Epoch 97/500
5210/5210 [==============================] - 2s 350us/sample - loss: 0.0030 - categorical_accuracy: 0.9994 - val_loss: 0.5488 - val_categorical_accuracy: 0.8572
Epoch 98/500
5210/5210 [==============================] - 2s 365us/sample - loss: 0.0250 - categorical_accuracy: 0.9939 - val_loss: 0.2001 - val_categorical_accuracy: 0.9482
Epoch 99/500
5210/5210 [==============================] - 2s 360us/sample - loss: 0.0053 - categorical_accuracy: 0.9990 - val_loss: 0.2222 - val_categorical_accuracy: 0.9545
Epoch 100/500
5210/5210 [==============================] - 2s 374us/sample - loss: 0.0016 - categorical_accuracy: 1.0000 - val_loss: 0.1001 - v

Epoch 142/500
5210/5210 [==============================] - 2s 361us/sample - loss: 0.0024 - categorical_accuracy: 0.9992 - val_loss: 0.4117 - val_categorical_accuracy: 0.9108
Epoch 143/500
5210/5210 [==============================] - 2s 354us/sample - loss: 0.0106 - categorical_accuracy: 0.9975 - val_loss: 0.2504 - val_categorical_accuracy: 0.9430
Epoch 144/500
5210/5210 [==============================] - 2s 361us/sample - loss: 0.0085 - categorical_accuracy: 0.9975 - val_loss: 0.1595 - val_categorical_accuracy: 0.9683
Epoch 145/500
5210/5210 [==============================] - 2s 367us/sample - loss: 0.0034 - categorical_accuracy: 0.9990 - val_loss: 0.1388 - val_categorical_accuracy: 0.9695
Epoch 146/500
5210/5210 [==============================] - 2s 355us/sample - loss: 0.0040 - categorical_accuracy: 0.9987 - val_loss: 0.1664 - val_categorical_accuracy: 0.9626
Epoch 147/500
5210/5210 [==============================] - 2s 368us/sample - loss: 0.0022 - categorical_accuracy: 0.9996 - va

5210/5210 [==============================] - 2s 346us/sample - loss: 0.0029 - categorical_accuracy: 0.9994 - val_loss: 1.1694 - val_categorical_accuracy: 0.7709
Epoch 189/500
5210/5210 [==============================] - 2s 343us/sample - loss: 0.0151 - categorical_accuracy: 0.9962 - val_loss: 0.1698 - val_categorical_accuracy: 0.9637
Epoch 190/500
5210/5210 [==============================] - 2s 338us/sample - loss: 0.0014 - categorical_accuracy: 1.0000 - val_loss: 0.1271 - val_categorical_accuracy: 0.9741
Epoch 191/500
5210/5210 [==============================] - 2s 343us/sample - loss: 6.9692e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1098 - val_categorical_accuracy: 0.9775
Epoch 192/500
5210/5210 [==============================] - 2s 351us/sample - loss: 9.4199e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1125 - val_categorical_accuracy: 0.9770
Epoch 193/500
5210/5210 [==============================] - 2s 368us/sample - loss: 4.1712e-04 - categorical_accuracy: 1.0000 - val_

...

Epoch 280/500
5210/5210 [==============================] - 2s 344us/sample - loss: 1.0420e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1114 - val_categorical_accuracy: 0.9845
Epoch 281/500
5210/5210 [==============================] - 2s 347us/sample - loss: 1.1208e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1177 - val_categorical_accuracy: 0.9781
Epoch 282/500
5210/5210 [==============================] - 2s 348us/sample - loss: 8.9547e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1223 - val_categorical_accuracy: 0.9827
Epoch 283/500
5210/5210 [==============================] - 2s 365us/sample - loss: 5.2395e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1219 - val_categorical_accuracy: 0.9804
Epoch 284/500
5210/5210 [==============================] - 2s 361us/sample - loss: 3.8396e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1160 - val_categorical_accuracy: 0.9793
Epoch 285/500
5210/5210 [==============================] - 2s 345us/sample - loss: 7.5309e-05 - categoric

Epoch 326/500
5210/5210 [==============================] - 2s 351us/sample - loss: 4.6355e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1260 - val_categorical_accuracy: 0.9799
Epoch 327/500
5210/5210 [==============================] - 2s 349us/sample - loss: 2.3955e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1224 - val_categorical_accuracy: 0.9804
Epoch 328/500
5210/5210 [==============================] - 2s 360us/sample - loss: 6.6265e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1216 - val_categorical_accuracy: 0.9827
Epoch 329/500
5210/5210 [==============================] - 2s 344us/sample - loss: 4.5177e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1190 - val_categorical_accuracy: 0.9816
Epoch 330/500
5210/5210 [==============================] - 2s 368us/sample - loss: 1.9956e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1167 - val_categorical_accuracy: 0.9827
Epoch 331/500
5210/5210 [==============================] - 2s 374us/sample - loss: 4.3017e-05 - categoric

Epoch 372/500
5210/5210 [==============================] - 2s 345us/sample - loss: 8.7064e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1043 - val_categorical_accuracy: 0.9839
Epoch 373/500
5210/5210 [==============================] - 2s 353us/sample - loss: 9.0244e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1024 - val_categorical_accuracy: 0.9839
Epoch 374/500
5210/5210 [==============================] - 2s 347us/sample - loss: 1.6043e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1053 - val_categorical_accuracy: 0.9839
Epoch 375/500
5210/5210 [==============================] - 2s 362us/sample - loss: 7.8592e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1053 - val_categorical_accuracy: 0.9827
Epoch 376/500
5210/5210 [==============================] - 2s 374us/sample - loss: 7.7654e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1070 - val_categorical_accuracy: 0.9833
Epoch 377/500
5210/5210 [==============================] - 2s 381us/sample - loss: 3.3263e-05 - categoric

Epoch 418/500
5210/5210 [==============================] - 2s 367us/sample - loss: 4.9323e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1130 - val_categorical_accuracy: 0.9839
Epoch 419/500
5210/5210 [==============================] - 2s 364us/sample - loss: 4.5404e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1120 - val_categorical_accuracy: 0.9839
Epoch 420/500
5210/5210 [==============================] - 2s 358us/sample - loss: 7.5315e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1125 - val_categorical_accuracy: 0.9822
Epoch 421/500
5210/5210 [==============================] - 2s 358us/sample - loss: 7.3742e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1129 - val_categorical_accuracy: 0.9833
Epoch 422/500
5210/5210 [==============================] - 2s 361us/sample - loss: 5.2624e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1136 - val_categorical_accuracy: 0.9839
Epoch 423/500
5210/5210 [==============================] - 2s 346us/sample - loss: 8.9153e-06 - categoric

Epoch 464/500
5210/5210 [==============================] - 2s 342us/sample - loss: 5.5484e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1138 - val_categorical_accuracy: 0.9833
Epoch 465/500
5210/5210 [==============================] - 2s 357us/sample - loss: 3.7849e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1126 - val_categorical_accuracy: 0.9839
Epoch 466/500
5210/5210 [==============================] - 2s 342us/sample - loss: 4.7033e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1130 - val_categorical_accuracy: 0.9827
Epoch 467/500
5210/5210 [==============================] - 2s 344us/sample - loss: 1.1127e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1157 - val_categorical_accuracy: 0.9810
Epoch 468/500
5210/5210 [==============================] - 2s 352us/sample - loss: 2.8712e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1177 - val_categorical_accuracy: 0.9822
Epoch 469/500
5210/5210 [==============================] - 2s 341us/sample - loss: 4.0141e-06 - categoric

In [76]:
metrics.accuracy_score(np.argmax(y_test, axis=-1),
                       np.argmax(model.predict(X_test), axis=-1))

0.9833045480713875

### Cross-day

In [77]:
X, y = read_cycle_session("session_2", normalize=False)

In [78]:
enc = OneHotEncoder()
enc.fit(y.reshape(-1,1))
y = enc.transform(y.reshape(-1, 1)).toarray()
X = X.reshape(-1, 128, 3)

In [79]:
y_pred = model.predict(X)
y_pred = np.argmax(y_pred, axis=1)

In [80]:
metrics.accuracy_score(np.argmax(y, axis=1), y_pred)

0.5952326116996776

## FCN normalized

### Same-day

In [81]:
X, y = read_cycle_session("session_1", normalize=True)

In [82]:
enc = OneHotEncoder()
enc.fit(y.reshape(-1,1))
y = enc.transform(y.reshape(-1, 1)).toarray()
X = X.reshape(-1, 128, 3)

In [83]:
cb, model = build_fcn((128, 3), 153, "zju_cycle_norm_fcn_500.hdf5")

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=RANDOM_STATE)

In [85]:
mini_batch_size = int(min(X_train.shape[0]/10, BATCH_SIZE))
start_time = time.time()
hist = model.fit(X_train, y_train, 
                      batch_size=mini_batch_size, 
                      epochs=EPOCHS,
                      verbose=True, 
                      validation_data=(X_val, y_val), 
                      callbacks=cb)
duration = time.time() - start_time
y_pred = model.predict(X_test)
keras.backend.clear_session()

Train on 5210 samples, validate on 1737 samples
Epoch 1/500
5210/5210 [==============================] - 3s 498us/sample - loss: 4.5099 - categorical_accuracy: 0.0846 - val_loss: 6.4517 - val_categorical_accuracy: 0.0063
Epoch 2/500
5210/5210 [==============================] - 2s 372us/sample - loss: 3.5398 - categorical_accuracy: 0.2739 - val_loss: 17.0919 - val_categorical_accuracy: 0.0058
Epoch 3/500
5210/5210 [==============================] - 2s 354us/sample - loss: 2.6152 - categorical_accuracy: 0.5152 - val_loss: 36.8134 - val_categorical_accuracy: 0.0086
Epoch 4/500
5210/5210 [==============================] - 2s 351us/sample - loss: 1.8035 - categorical_accuracy: 0.7230 - val_loss: 28.3517 - val_categorical_accuracy: 0.0161
Epoch 5/500
5210/5210 [==============================] - 2s 358us/sample - loss: 1.2100 - categorical_accuracy: 0.8478 - val_loss: 46.7413 - val_categorical_accuracy: 0.0069
Epoch 6/500
5210/5210 [==============================] - 2s 362us/sample - loss: 0.

5210/5210 [==============================] - 2s 374us/sample - loss: 0.0065 - categorical_accuracy: 0.9992 - val_loss: 1.4698 - val_categorical_accuracy: 0.6644
Epoch 48/500
5210/5210 [==============================] - 2s 366us/sample - loss: 0.0342 - categorical_accuracy: 0.9914 - val_loss: 41.6797 - val_categorical_accuracy: 0.0150
Epoch 49/500
5210/5210 [==============================] - 2s 363us/sample - loss: 0.0285 - categorical_accuracy: 0.9942 - val_loss: 19.7168 - val_categorical_accuracy: 0.0933
Epoch 50/500
5210/5210 [==============================] - 2s 361us/sample - loss: 0.0126 - categorical_accuracy: 0.9985 - val_loss: 20.3383 - val_categorical_accuracy: 0.0823
Epoch 51/500
5210/5210 [==============================] - 2s 362us/sample - loss: 0.0109 - categorical_accuracy: 0.9981 - val_loss: 4.7913 - val_categorical_accuracy: 0.2930
Epoch 52/500
5210/5210 [==============================] - 2s 365us/sample - loss: 0.0046 - categorical_accuracy: 0.9998 - val_loss: 1.2175 -

Epoch 94/500
5210/5210 [==============================] - 2s 361us/sample - loss: 0.0119 - categorical_accuracy: 0.9985 - val_loss: 7.3040 - val_categorical_accuracy: 0.2412
Epoch 95/500
5210/5210 [==============================] - 2s 367us/sample - loss: 0.0019 - categorical_accuracy: 0.9996 - val_loss: 0.1235 - val_categorical_accuracy: 0.9729
Epoch 96/500
5210/5210 [==============================] - 2s 365us/sample - loss: 0.0010 - categorical_accuracy: 1.0000 - val_loss: 0.1054 - val_categorical_accuracy: 0.9799
Epoch 97/500
5210/5210 [==============================] - 2s 349us/sample - loss: 7.5899e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1055 - val_categorical_accuracy: 0.9787
Epoch 98/500
5210/5210 [==============================] - 2s 360us/sample - loss: 0.0183 - categorical_accuracy: 0.9956 - val_loss: 67.3262 - val_categorical_accuracy: 0.0069
Epoch 99/500
5210/5210 [==============================] - 2s 351us/sample - loss: 0.0268 - categorical_accuracy: 0.9923 - val

5210/5210 [==============================] - 2s 367us/sample - loss: 6.3097e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1234 - val_categorical_accuracy: 0.9758
Epoch 141/500
5210/5210 [==============================] - 2s 356us/sample - loss: 4.6574e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1257 - val_categorical_accuracy: 0.9729
Epoch 142/500
5210/5210 [==============================] - 2s 353us/sample - loss: 5.0311e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1128 - val_categorical_accuracy: 0.9770
Epoch 143/500
5210/5210 [==============================] - 2s 366us/sample - loss: 3.2538e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1156 - val_categorical_accuracy: 0.9764
Epoch 144/500
5210/5210 [==============================] - 2s 369us/sample - loss: 2.7385e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1106 - val_categorical_accuracy: 0.9787
Epoch 145/500
5210/5210 [==============================] - 2s 386us/sample - loss: 2.4335e-04 - categorical_accuracy: 1

5210/5210 [==============================] - 2s 369us/sample - loss: 0.0017 - categorical_accuracy: 0.9996 - val_loss: 0.1511 - val_categorical_accuracy: 0.9695
Epoch 187/500
5210/5210 [==============================] - 2s 352us/sample - loss: 0.0105 - categorical_accuracy: 0.9975 - val_loss: 3.9372 - val_categorical_accuracy: 0.5170
Epoch 188/500
5210/5210 [==============================] - 2s 350us/sample - loss: 0.0101 - categorical_accuracy: 0.9969 - val_loss: 10.3235 - val_categorical_accuracy: 0.2343
Epoch 189/500
5210/5210 [==============================] - 2s 353us/sample - loss: 0.0030 - categorical_accuracy: 0.9990 - val_loss: 0.2940 - val_categorical_accuracy: 0.9338
Epoch 190/500
5210/5210 [==============================] - 2s 350us/sample - loss: 6.5017e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1327 - val_categorical_accuracy: 0.9770
Epoch 191/500
5210/5210 [==============================] - 2s 353us/sample - loss: 6.7720e-04 - categorical_accuracy: 1.0000 - val_los

...

Epoch 278/500
5210/5210 [==============================] - 2s 358us/sample - loss: 5.6614e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1266 - val_categorical_accuracy: 0.9833
Epoch 279/500
5210/5210 [==============================] - 2s 351us/sample - loss: 9.8776e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1141 - val_categorical_accuracy: 0.9816
Epoch 280/500
5210/5210 [==============================] - 2s 361us/sample - loss: 7.9295e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1141 - val_categorical_accuracy: 0.9833
Epoch 281/500
5210/5210 [==============================] - 2s 354us/sample - loss: 7.9586e-04 - categorical_accuracy: 0.9998 - val_loss: 0.6757 - val_categorical_accuracy: 0.8659
Epoch 282/500
5210/5210 [==============================] - 2s 341us/sample - loss: 0.0022 - categorical_accuracy: 0.9994 - val_loss: 0.6360 - val_categorical_accuracy: 0.8768
Epoch 283/500
5210/5210 [==============================] - 2s 346us/sample - loss: 0.0014 - categorical_accur

Epoch 324/500
5210/5210 [==============================] - 2s 354us/sample - loss: 3.7598e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1236 - val_categorical_accuracy: 0.9827
Epoch 325/500
5210/5210 [==============================] - 2s 353us/sample - loss: 1.4386e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1365 - val_categorical_accuracy: 0.9787
Epoch 326/500
5210/5210 [==============================] - 2s 343us/sample - loss: 0.0013 - categorical_accuracy: 0.9998 - val_loss: 0.1646 - val_categorical_accuracy: 0.9729
Epoch 327/500
5210/5210 [==============================] - 2s 340us/sample - loss: 2.6519e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1339 - val_categorical_accuracy: 0.9799
Epoch 328/500
5210/5210 [==============================] - 2s 343us/sample - loss: 5.7520e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1315 - val_categorical_accuracy: 0.9827
Epoch 329/500
5210/5210 [==============================] - 2s 359us/sample - loss: 4.0063e-05 - categorical_a

Epoch 370/500
5210/5210 [==============================] - 2s 354us/sample - loss: 1.1835e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1187 - val_categorical_accuracy: 0.9833
Epoch 371/500
5210/5210 [==============================] - 2s 355us/sample - loss: 7.2432e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1205 - val_categorical_accuracy: 0.9839
Epoch 372/500
5210/5210 [==============================] - 2s 361us/sample - loss: 6.3352e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1220 - val_categorical_accuracy: 0.9839
Epoch 373/500
5210/5210 [==============================] - 2s 365us/sample - loss: 6.7192e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1187 - val_categorical_accuracy: 0.9827
Epoch 374/500
5210/5210 [==============================] - 2s 363us/sample - loss: 9.9853e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1138 - val_categorical_accuracy: 0.9822
Epoch 375/500
5210/5210 [==============================] - 2s 374us/sample - loss: 7.9098e-06 - categoric

Epoch 416/500
5210/5210 [==============================] - 2s 354us/sample - loss: 3.6364e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1206 - val_categorical_accuracy: 0.9839
Epoch 417/500
5210/5210 [==============================] - 2s 360us/sample - loss: 3.7581e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1225 - val_categorical_accuracy: 0.9850
Epoch 418/500
5210/5210 [==============================] - 2s 362us/sample - loss: 4.3810e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1184 - val_categorical_accuracy: 0.9850
Epoch 419/500
5210/5210 [==============================] - 2s 356us/sample - loss: 3.1178e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1188 - val_categorical_accuracy: 0.9839
Epoch 420/500
5210/5210 [==============================] - 2s 356us/sample - loss: 1.2867e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1226 - val_categorical_accuracy: 0.9833
Epoch 421/500
5210/5210 [==============================] - 2s 355us/sample - loss: 3.8890e-06 - categoric

Epoch 462/500
5210/5210 [==============================] - 2s 368us/sample - loss: 1.6411e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1197 - val_categorical_accuracy: 0.9845
Epoch 463/500
5210/5210 [==============================] - 2s 369us/sample - loss: 1.8317e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1199 - val_categorical_accuracy: 0.9850
Epoch 464/500
5210/5210 [==============================] - 2s 377us/sample - loss: 1.3728e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1180 - val_categorical_accuracy: 0.9845
Epoch 465/500
5210/5210 [==============================] - 2s 373us/sample - loss: 1.9341e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1177 - val_categorical_accuracy: 0.9845
Epoch 466/500
5210/5210 [==============================] - 2s 365us/sample - loss: 1.6224e-06 - categorical_accuracy: 1.0000 - val_loss: 0.1187 - val_categorical_accuracy: 0.9850
Epoch 467/500
5210/5210 [==============================] - 2s 369us/sample - loss: 1.9397e-06 - categoric

In [86]:
metrics.accuracy_score(np.argmax(y_test, axis=-1),
                       np.argmax(model.predict(X_test), axis=-1))

0.9844559585492227

### Cross-day

In [87]:
X, y = read_cycle_session("session_2", normalize=True)

In [88]:
enc = OneHotEncoder()
enc.fit(y.reshape(-1,1))
y = enc.transform(y.reshape(-1, 1)).toarray()
X = X.reshape(-1, 128, 3)

In [89]:
y_pred = model.predict(X)
y_pred = np.argmax(y_pred, axis=1)

In [90]:
metrics.accuracy_score(np.argmax(y, axis=1), y_pred)

0.5988023952095808